In [1]:
## TO STACK ALL THE IMAGES IN ONE WINDOW
def stackImages(imgArray,scale,lables=[]):
    rows = len(imgArray)
    cols = len(imgArray[0])
    rowsAvailable = isinstance(imgArray[0], list)
    width = imgArray[0][0].shape[1]
    height = imgArray[0][0].shape[0]
    if rowsAvailable:
        for x in range ( 0, rows):
            for y in range(0, cols):
                imgArray[x][y] = cv2.resize(imgArray[x][y], (0, 0), None, scale, scale)
                if len(imgArray[x][y].shape) == 2: imgArray[x][y]= cv2.cvtColor( imgArray[x][y], cv2.COLOR_GRAY2BGR)
        imageBlank = np.zeros((height, width, 3), np.uint8)
        hor = [imageBlank]*rows
        hor_con = [imageBlank]*rows
        for x in range(0, rows):
            hor[x] = np.hstack(imgArray[x])
            hor_con[x] = np.concatenate(imgArray[x])
        ver = np.vstack(hor)
        ver_con = np.concatenate(hor)
    else:
        for x in range(0, rows):
            imgArray[x] = cv2.resize(imgArray[x], (0, 0), None, scale, scale)
            if len(imgArray[x].shape) == 2: imgArray[x] = cv2.cvtColor(imgArray[x], cv2.COLOR_GRAY2BGR)
        hor= np.hstack(imgArray)
        hor_con= np.concatenate(imgArray)
        ver = hor
    if len(lables) != 0:
        eachImgWidth= int(ver.shape[1] / cols)
        eachImgHeight = int(ver.shape[0] / rows)
        #print(eachImgHeight)
        for d in range(0, rows):
            for c in range (0,cols):
                cv2.rectangle(ver,(c*eachImgWidth,eachImgHeight*d),(c*eachImgWidth+len(lables[d])*13+27,30+eachImgHeight*d),(255,255,255),cv2.FILLED)
                cv2.putText(ver,lables[d],(eachImgWidth*c+10,eachImgHeight*d+20),cv2.FONT_HERSHEY_COMPLEX,0.7,(255,0,255),2)
    return ver

In [2]:
def getCornerPoints(cont):
    peri = cv2.arcLength(cont, True) # LENGTH OF CONTOUR
    approx = cv2.approxPolyDP(cont, 0.02 * peri, True) # APPROXIMATE THE POLY TO GET CORNER POINTS
    return approx

In [3]:
def rectContour(contours):
 
    rectCon = []
    max_area = 0
    for i in contours:
        area = cv2.contourArea(i)
        if area > 50:
            peri = cv2.arcLength(i, True)
            approx = cv2.approxPolyDP(i, 0.02 * peri, True)
            if len(approx) == 4:
                rectCon.append(i)
    rectCon = sorted(rectCon, key=cv2.contourArea,reverse=True)
    #print(len(rectCon))
    return rectCon

In [4]:
def reorder(myPoints):
 
    myPoints = myPoints.reshape((4, 2)) # REMOVE EXTRA BRACKET
    print(myPoints)
    myPointsNew = np.zeros((4, 1, 2), np.int32) # NEW MATRIX WITH ARRANGED POINTS
    add = myPoints.sum(1)
    print(add)
    print(np.argmax(add))
    myPointsNew[0] = myPoints[np.argmin(add)]  #[0,0]
    myPointsNew[3] =myPoints[np.argmax(add)]   #[w,h]
    diff = np.diff(myPoints, axis=1)
    myPointsNew[1] =myPoints[np.argmin(diff)]  #[w,0]
    myPointsNew[2] = myPoints[np.argmax(diff)] #[h,0]
 
    return myPointsNew

In [5]:
###########For Name
def hsplitBoxes(img,CN,RN):
    cols = np.hsplit(img,CN)
    boxes=[]
    for r in cols:
        rows= np.vsplit(r,RN)
        for box in rows:
            boxes.append(box)
        
    return boxes



In [153]:
def vsplitAnsBoxes(img,CN,RN):
    rows = np.vsplit(img,RN)
    boxes=[]
    for r in rows:
        cols= np.hsplit(r,CN)
        for box in cols:
            boxes.append(box)
            
    return boxes

In [154]:
def vsplitBoxes(img):
    rows = np.vsplit(img,26)
    boxes=[]
    for r in rows:
        boxes.append(r)
            
    return boxes

In [155]:
def showAnswers(image,myIndex,grading,ans,questions=5,choices=5):
    secW = int(image.shape[1]/questions)
    secH = int(image.shape[0]/choices)
 
    for x in range(0,questions):
        myAns= myIndex[x]
        cX = (myAns * secW) + secW // 2
        cY = (x * secH) + secH // 2
        if grading[x]==1:
            myColor = (0,255,0)
            #cv2.rectangle(img,(myAns*secW,x*secH),((myAns*secW)+secW,(x*secH)+secH),myColor,cv2.FILLED)
            cv2.circle(image,(cX,cY),50,myColor,cv2.FILLED)
        else:
            myColor = (0,0,255)
            #cv2.rectangle(img, (myAns * secW, x * secH), ((myAns * secW) + secW, (x * secH) + secH), myColor, cv2.FILLED)
            cv2.circle(image,(cX,cY),50,myColor,cv2.FILLED)
            # CORRECT ANSWER
            myColor = (0, 255, 0)
            correctAns = ans[x]
            cv2.circle(image,((correctAns * secW)+secW//2, (x * secH)+secH//2),20,myColor,cv2.FILLED)
        
    return image

In [156]:
def drawGrid(img,questions=5,choices=5):
    secW = int(img.shape[1]/questions)
    secH = int(img.shape[0]/choices)
    for i in range (0,9):
        pt1 = (0,secH*i)
        pt2 = (img.shape[1],secH*i)
        pt3 = (secW * i, 0)
        pt4 = (secW*i,img.shape[0])
        cv2.line(img, pt1, pt2, (255, 255, 0),2)
        cv2.line(img, pt3, pt4, (255, 255, 0),2)
 
    return img

In [157]:
#convert list to string
def listToString(s):  
    
    # initialize an empty string 
    str1 = ""  
    
    # traverse in the string   
    for ele in s:  
        str1 += ele   
    
    # return string   
    return str1  

In [332]:
def markGrades(name,roll,mobile,setcode,total,grade):
    with open('grades.csv','r+') as f:
        myDataList = f.readlines()
        nameList =[]
        for line in myDataList:
            entry = line.split(',')
            nameList.append(entry[0])
        if name not in  line:
          
            f.writelines(f'\n{name},{roll},{mobile},{setcode},{total},{grade}')

In [333]:
import cv2
import numpy as np


In [429]:
img=cv2.imread('3.jpg')

In [430]:
#parameters
width_img=700
height_img=700


answers=[3,0,3,1,3,0,1,2,2,1,1,3,0,1,2,1,2,3,1,2,2,0,0,3,1,3,0,2,1,0]

In [431]:
img=cv2.resize(img,(width_img,height_img))

In [432]:
# preprocess our image        #10  50
imgBlank = np.zeros_like(img)
imgGray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
imgBlur = cv2.GaussianBlur(imgGray, (5, 5), 1) # ADD GAUSSIAN BLUR
imgCanny = cv2.Canny(imgBlur,50,220) # APPLY CANNY # detcting edges of the image

In [433]:
## FIND ALL COUNTOURS
imgContours = img.copy()

contours, hierarchy = cv2.findContours(imgCanny, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) # FIND ALL CONTOURS
cv2.drawContours(imgContours, contours, -1, (0, 255, 0), 5) # DRAW ALL DETECTED CONTOURS

array([[[250, 253, 255],
        [251, 255, 255],
        [252, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[252, 253, 255],
        [253, 255, 255],
        [254, 255, 254],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[254, 254, 254],
        [255, 255, 254],
        [255, 255, 254],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]]

In [434]:
#dont need of small rectangles and small area coutours
rectCons=rectContour(contours)   #fine all rect contour in our images
nameContour=getCornerPoints(rectCons[0])  #for first biggest rectangle
rollNumber=getCornerPoints(rectCons[7])#for forth biggest rectangle
mobileNumber=getCornerPoints(rectCons[6])# for second biggest rectangle
answers1=getCornerPoints(rectCons[5])
answers2=getCornerPoints(rectCons[2])
answers3=getCornerPoints(rectCons[3])
answers4=getCornerPoints(rectCons[4])
answers5=getCornerPoints(rectCons[1])
setCode=getCornerPoints(rectCons[10])

In [435]:
print(len(rollNumber))

4


In [436]:
imgBiggestContour = img.copy()
if nameContour.size != 0 and rollNumber.size != 0:
    cv2.drawContours(imgBiggestContour, nameContour, -1, (0, 255, 0), 10) 
    cv2.drawContours(imgBiggestContour,rollNumber , -1, (255, 0, 0), 10)
    cv2.drawContours(imgBiggestContour,mobileNumber , -1, (255, 0, 0), 10)
    cv2.drawContours(imgBiggestContour,answers1 , -1, (255, 0, 0), 10)
    cv2.drawContours(imgBiggestContour,answers2 , -1, (255, 0, 0), 10)
    cv2.drawContours(imgBiggestContour,answers3 , -1, (255, 0, 0), 10)
    cv2.drawContours(imgBiggestContour,answers4 , -1, (255, 0, 0), 10)
    cv2.drawContours(imgBiggestContour,answers5 , -1, (255, 0, 0), 10)
    cv2.drawContours(imgBiggestContour,setCode , -1, (0, 0, 255), 10)
    nameContour=reorder(nameContour)
    rollNumber=reorder(rollNumber)
    mobileNumber=reorder(mobileNumber)
    answers1=reorder(answers1)
    answers2=reorder(answers2)
    answers3=reorder(answers3)
    answers4=reorder(answers4)
    answers4=reorder(answers5)
    setCode=reorder(setCode)
############################################################################################################################
###########################################################################################################################
################################################################################################################################


    ###############################################@###########
    namepts1 = np.float32(nameContour) # PREPARE POINTS FOR WARP
    namepts2 = np.float32([[0, 0],[width_img, 0], [0, height_img],[width_img, height_img]]) # PREPARE POINTS FOR WARP
    matrix = cv2.getPerspectiveTransform(namepts1, namepts2) # GET TRANSFORMATION MATRIX
    nameWarpColored = cv2.warpPerspective(img, matrix, (width_img, height_img)) # APPLY WARP PERSPECTIVE
    
    # APPLY THRESHOLD
    nameWarpGray = cv2.cvtColor(nameWarpColored,cv2.COLOR_BGR2GRAY) # CONVERT TO GRAYSCALE
    nameThresh = cv2.threshold(nameWarpGray, 170, 245,cv2.THRESH_BINARY_INV )[1] # APPLY THRESHOLD AND INVERSE
    nameCropped = nameThresh[93:width_img-9,10:height_img-10]
    nameboxes = hsplitBoxes(nameCropped,34,26) # GET INDIVIDUAL BOXES

    
    
#     cv2.imshow("Split Test ", boxes[19])
#     cv2.waitKey(0)
#Getting nonZero pixel values of each box
    nameCol=34
    nameRow=26
    countR=0
    countC=0
    namePixelVal = np.zeros((nameCol,nameRow)) # TO STORE THE NON ZERO VALUES OF EACH BOX
    for image in nameboxes:
        #cv2.imshow(str(countR)+str(countC),image)
        totalnamePixels = cv2.countNonZero(image)
        namePixelVal[countR][countC]= totalnamePixels
        countC += 1
        if (countC==nameRow):countC=0;countR +=1
            
 # FIND THE USER ANSWERS AND PUT THEM IN A LIST
    myNameIndex=[]
    for x in range (0,nameCol):
        arr = namePixelVal[x]
        nameIndexVal = np.where(arr == np.amax(arr))
        if np.amax(arr)>200:
            myNameIndex.append(nameIndexVal[0][0])
        else:
            myNameIndex.append(' ')
            
#     print(myNameIndex)      
         
################################################################################################################################
###############################################################################################################################
##############################################################################################################################


# #     print(cv2.countNonZero(boxes[1]),cv2.countNonZero(boxes[2]))
    
    ##############################################################
    rollpts1 = np.float32(rollNumber) # PREPARE POINTS FOR WARP
    rollpts2 = np.float32([[0, 0],[width_img, 0], [0, height_img],[width_img, height_img]]) # PREPARE POINTS FOR WARP
    matrix = cv2.getPerspectiveTransform(rollpts1, rollpts2) # GET TRANSFORMATION MATRIX
    rollWarpColored = cv2.warpPerspective(img, matrix, (width_img, height_img)) # APPLY WARP PERSPECTIVE
    
        # APPLY THRESHOLD
    rollWarpGray = cv2.cvtColor(rollWarpColored,cv2.COLOR_BGR2GRAY) # CONVERT TO GRAYSCALE
    rollThresh = cv2.threshold(rollWarpGray, 170, 245,cv2.THRESH_BINARY_INV )[1] # APPLY THRESHOLD AND INVERSE
    rollCropped = rollThresh[203:width_img-17,14:height_img-14]
    rollboxes = hsplitBoxes(rollCropped,7,10) # GET INDIVIDUAL BOXES
    
#     cv2.imshow("Split Test ", rollboxes[17])
#     cv2.waitKey(0)
    #Getting nonZero pixel values of each box
    rollCol=7
    rollRow=10
    countR=0
    countC=0
    rollPixelVal = np.zeros((rollCol,rollRow)) # TO STORE THE NON ZERO VALUES OF EACH BOX
    for image in rollboxes:
        #cv2.imshow(str(countR)+str(countC),image)
        totalrollPixels = cv2.countNonZero(image)
        rollPixelVal[countR][countC]= totalrollPixels
        countC += 1
        if (countC==rollRow):countC=0;countR +=1
    # FIND THE USER ANSWERS AND PUT THEM IN A LIST
    myRollIndex=[]
    for x in range (0,rollCol):
        arr = rollPixelVal[x]
        rollIndexVal = np.where(arr == np.amax(arr))
        myRollIndex.append(rollIndexVal[0][0])
#     print(myRollIndex)        
##############################################################################################################################
###############################################################################################################################
###############################################################################################################################
    
    ##############################################################
    mobilepts1 = np.float32(mobileNumber) # PREPARE POINTS FOR WARP
    mobilepts2 = np.float32([[0, 0],[width_img, 0], [0, height_img],[width_img, height_img]]) # PREPARE POINTS FOR WARP
    matrix = cv2.getPerspectiveTransform(mobilepts1, mobilepts2) # GET TRANSFORMATION MATRIX
    mobileWarpColored = cv2.warpPerspective(img, matrix, (width_img, height_img)) # APPLY WARP PERSPECTIVE
    
        # APPLY THRESHOLD
    mobileWarpGray = cv2.cvtColor(mobileWarpColored,cv2.COLOR_BGR2GRAY) # CONVERT TO GRAYSCALE
    mobileThresh = cv2.threshold(mobileWarpGray, 170, 245,cv2.THRESH_BINARY_INV )[1] # APPLY THRESHOLD AND INVERSE
    mobileCropped = mobileThresh[213:width_img-17,15:height_img-15]
    mobileboxes = hsplitBoxes(mobileCropped,10,10) # GET INDIVIDUAL BOXES
    
#     cv2.imshow("Split Test ", mobileCropped)
#     cv2.waitKey(0)
    #Getting nonZero pixel values of each box
    mobileCol=10
    mobileRow=10
    countR=0
    countC=0
    mobilePixelVal = np.zeros((mobileCol,mobileRow)) # TO STORE THE NON ZERO VALUES OF EACH BOX
    for image in mobileboxes:
        #cv2.imshow(str(countR)+str(countC),image)
        totalmobilePixels = cv2.countNonZero(image)
        mobilePixelVal[countR][countC]= totalmobilePixels
        countC += 1
        if (countC==mobileRow):countC=0;countR +=1
    # FIND THE USER ANSWERS AND PUT THEM IN A LIST
    myMobileIndex=[]
    for x in range (0,mobileCol):
        arr = mobilePixelVal[x]
        mobileIndexVal = np.where(arr == np.amax(arr))
        myMobileIndex.append(mobileIndexVal[0][0])
#     print(myRollIndex)        

###############################################################################################################################
###############################################################################################################################
###############################################################################################################################



    ########################333####################################
    ans1pts1 = np.float32(answers1) # PREPARE POINTS FOR WARP
    ans1pts2 = np.float32([[0, 0],[width_img, 0], [0, height_img],[width_img, height_img]]) # PREPARE POINTS FOR WARP
    matrix = cv2.getPerspectiveTransform(ans1pts1, ans1pts2) # GET TRANSFORMATION MATRIX
    ans1WarpColored = cv2.warpPerspective(img, matrix, (width_img, height_img)) # APPLY WARP PERSPECTIVE
    
    
            # APPLY THRESHOLD
    ans1WarpGray = cv2.cvtColor(ans1WarpColored,cv2.COLOR_BGR2GRAY) # CONVERT TO GRAYSCALE
    ans1Thresh = cv2.threshold(ans1WarpGray, 160, 245,cv2.THRESH_BINARY_INV )[1] # APPLY THRESHOLD AND INVERSE
    ans1Cropped = ans1Thresh[4:width_img-0,149:height_img-19]
    ans1boxes = hsplitBoxes(ans1Cropped,1,8) # GET INDIVIDUAL BOXES
    myAns1Index=[]
    
    for i in range(0,8):
        ans1CroppedAgain = ans1boxes[i][0:80,0:height_img]
        ans1boxesAgain=vsplitAnsBoxes(ans1CroppedAgain,4,5)




        #Getting nonZero pixel values of each box
        ans1Col=4
        ans1Row=5
        countR=0
        countC=0
        ans1PixelVal = np.zeros((ans1Row,ans1Col)) # TO STORE THE NON ZERO VALUES OF EACH BOX
        for image in ans1boxesAgain:
            #cv2.imshow(str(countR)+str(countC),image)
            totalans1Pixels = cv2.countNonZero(image)
#             print(cv2.countNonZero(image))
            ans1PixelVal[countR][countC]= totalans1Pixels
            countC += 1
            if (countC==ans1Row-1):countC=0;countR +=1
        # FIND THE USER ANSWERS AND PUT THEM IN A LIST

        for x in range (0,ans1Row):
            arr = ans1PixelVal[x]
            ans1IndexVal = np.where(arr == np.amax(arr))
#             myAns1Index.append(ans1IndexVal[0][0])    

            if np.amax(arr)>400:
                myAns1Index.append(ans1IndexVal[0][0])
            else:
                myAns1Index.append(' ')



#     cv2.imshow("Split Test ",ans1boxesAgain[11] )
#     cv2.waitKey(0)


#################################################################################################################################
##############################################################################################################################
############################################################################################################################



#     ################################################################
#     ans2pts1 = np.float32(answers2) # PREPARE POINTS FOR WARP
#     ans2pts2 = np.float32([[0, 0],[width_img, 0], [0, height_img],[width_img, height_img]]) # PREPARE POINTS FOR WARP
#     matrix = cv2.getPerspectiveTransform(ans2pts1, ans2pts2) # GET TRANSFORMATION MATRIX
#     ans2WarpColored = cv2.warpPerspective(img, matrix, (width_img, height_img)) # APPLY WARP PERSPECTIVE
    
###############################################################################################################################
###############################################################################################################################
###############################################################################################################################


#     ##########################################################
#     ans3pts1 = np.float32(answers3) # PREPARE POINTS FOR WARP
#     ans3pts2 = np.float32([[0, 0],[width_img, 0], [0, height_img],[width_img, height_img]]) # PREPARE POINTS FOR WARP
#     matrix = cv2.getPerspectiveTransform(ans3pts1, ans3pts2) # GET TRANSFORMATION MATRIX
#     ans3WarpColored = cv2.warpPerspective(img, matrix, (width_img, height_img)) # APPLY WARP PERSPECTIVE


###############################################################################################################################
###############################################################################################################################
###############################################################################################################################


    
#     ###########################################################
#     ans4pts1 = np.float32(answers4) # PREPARE POINTS FOR WARP
#     ans4pts2 = np.float32([[0, 0],[width_img, 0], [0, height_img],[width_img, height_img]]) # PREPARE POINTS FOR WARP
#     matrix = cv2.getPerspectiveTransform(ans4pts1, ans4pts2) # GET TRANSFORMATION MATRIX
#     ans4WarpColored = cv2.warpPerspective(img, matrix, (width_img, height_img)) # APPLY WARP PERSPECTIVE


###############################################################################################################################
###############################################################################################################################
##############################################################################################################################
    
    ##########################################################
    setCodepts1 = np.float32(setCode) # PREPARE POINTS FOR WARP
    setCode4pts2 = np.float32([[0, 0],[width_img, 0], [0, height_img],[width_img, height_img]]) # PREPARE POINTS FOR WARP
    matrix = cv2.getPerspectiveTransform(setCodepts1, setCode4pts2) # GET TRANSFORMATION MATRIX
    setCodeWarpColored = cv2.warpPerspective(img, matrix, (width_img, height_img)) # APPLY WARP PERSPECTIVE
    
        # APPLY THRESHOLD
    setCodeWarpGray = cv2.cvtColor(setCodeWarpColored,cv2.COLOR_BGR2GRAY) # CONVERT TO GRAYSCALE
    setCodeThresh = cv2.threshold(setCodeWarpGray, 170, 245,cv2.THRESH_BINARY_INV )[1] # APPLY THRESHOLD AND INVERSE
    setCodeCropped = setCodeThresh[105:width_img-3,210:height_img-240]
    setCodeboxes = hsplitBoxes(setCodeCropped,1,4) # GET INDIVIDUAL BOXES

#     cv2.imshow("Split Test ", setCodeCropped)
#     cv2.waitKey(0)

#Getting nonZero pixel values of each box
    setCodeCol=1
    setCodeRow=4
    countR=0
    countC=0
    setCodePixelVal = np.zeros((setCodeCol,setCodeRow)) # TO STORE THE NON ZERO VALUES OF EACH BOX
    for image in setCodeboxes:
        #cv2.imshow(str(countR)+str(countC),image)
        totalsetCodePixels = cv2.countNonZero(image)
        setCodePixelVal[countR][countC]= totalsetCodePixels
        countC += 1
        if (countC==setCodeRow):countC=0;countR +=1
            
 # FIND THE USER ANSWERS AND PUT THEM IN A LIST
    mySetCodeIndex=[]
    for x in range (0,setCodeCol):
        arr = setCodePixelVal[x]
        setCodeIndexVal = np.where(arr == np.amax(arr))
        if np.amax(arr)>200:
            mySetCodeIndex.append(setCodeIndexVal[0][0])
        else:
            mySetCodeIndex.append(' ')



[[  7   6]
 [  7 272]
 [426 272]
 [426   6]]
[ 13 279 698 432]
2
[[441   6]
 [441 133]
 [576 132]
 [575   6]]
[447 574 708 581]
2
[[518 141]
 [518 272]
 [692 272]
 [692 141]]
[659 790 964 833]
2
[[  7 280]
 [  7 694]
 [138 694]
 [139 281]]
[287 701 832 420]
2
[[147 280]
 [145 693]
 [277 694]
 [277 281]]
[427 838 971 558]
2
[[286 280]
 [284 694]
 [415 694]
 [416 281]]
[ 566  978 1109  697]
2
[[423 280]
 [422 693]
 [554 694]
 [554 281]]
[ 703 1115 1248  835]
2
[[563 280]
 [561 694]
 [692 694]
 [693 281]]
[ 843 1255 1386  974]
2
[[586   5]
 [582 132]
 [620 133]
 [621   8]]
[591 714 753 629]
2


### For Name

In [437]:
# myNameIndex

In [438]:
####   For Name  #####################

name_list=[]
alpha = 'A'

for i in range(0, 40):
    if myNameIndex[i] != ' ':
        z=int(myNameIndex[i])
        name_list.append(chr(ord(alpha) +  z))
        j=0
    else:
        
        j=j+1
        if j>1:

            break
        else:
            name_list.append(' ')

#     alpha = chr(ord(alpha) + myNameIndex[i]) 

In [439]:
name_list

['H', 'A', 'R', 'S', 'H', 'I', 'T', ' ']

In [440]:
name_list=name_list[0:len(name_list)-1]

In [441]:
name_list

['H', 'A', 'R', 'S', 'H', 'I', 'T']

In [442]:
Name=listToString(name_list)

In [443]:
Name

'HARSHIT'

### For Roll Number

In [444]:
######## for roll number  ###############
myRollIndex

for i in range(0,len(myRollIndex)):
    
    if myRollIndex[i] == 9:
        myRollIndex[i]= -1
        



In [445]:
# add 1 to each list element
myRollIndex = [x + 1 for x in myRollIndex] 



In [446]:
# convert list to string
RollStr = [str(myRollIndex) for myRollIndex in myRollIndex]
# RollStr
Roll= "".join(RollStr)

In [447]:
Roll

'8405010'

### For Mobile Number 

In [448]:
for i in range(0,len(myMobileIndex)):
    
    if myMobileIndex[i] == 9:
        myMobileIndex[i]= -1
        

In [449]:
# add 1 to each list element
myMobileIndex = [x + 1 for x in myMobileIndex] 


In [450]:
# convert list to string
MobileStr = [str(myMobileIndex) for myMobileIndex in myMobileIndex]
# RollStr
Mobile_number= "".join(MobileStr)

In [451]:
Mobile_number

'9927756721'

### For SetCode 

In [452]:
SetCode=mySetCodeIndex[0]+1

In [453]:
SetCode

2

### For Answer Column 1

In [454]:
len(myAns1Index)

40

In [455]:
k=0
T=4
NULL=0
F=-1
for Q in myAns1Index:
    if k < 30:
        if Q == answers[k] and Q != ' ':
            myAns1Index[k]=T    # if answer is True
        elif Q != answers[k] and Q != ' ':
            myAns1Index[k]=F     # if answer is wrong
        else:
            myAns1Index[k]=NULL  #Not attempting
        k=k+1    
        

In [456]:

myAns1Index

[-1,
 -1,
 0,
 4,
 4,
 4,
 -1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 -1,
 0,
 0,
 0,
 4,
 0,
 4,
 4,
 4,
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ']

### Grading 

In [457]:
total = 0
for ele in range(0, 30): 
    total = total + myAns1Index[ele] 
    
total    

24

In [458]:
N_Quetion=30
each_Qution_Value=4
grade=total/(N_Quetion*each_Qution_Value)*100

In [459]:
grade

20.0

### write in CSV

In [461]:
markGrades(Name,Roll,Mobile_number,SetCode,total,grade)

### checking Height and width for Spliting Image in horizontaly or vertically

In [75]:
# height, width = nameCropped.shape[:2]
# height, width = rollCropped.shape[:2]
# height, width=mobileCropped.shape[:2]
# height, width =setCodeCropped.shape[:2]
# height, width = ans1Cropped.shape[:2]

# height,width=ans1CroppedAgain.shape[:2]

In [76]:
height,width

(696, 532)

In [402]:
# cv2.imshow('dfdscked imgs',rollCropped)
# cv2.waitKey(0)

-1

In [464]:

# boxes = vsplitBoxes(nameCropped) # GET INDIVIDUAL BOXES
        
# cv2.imshow('dfsd',nameCropped)
# cv2.waitKey(0)

In [465]:
imgArray=([img,imgGray,imgBlur,imgCanny],
          [imgContours,imgBiggestContour,rollWarpColored,rollThresh],
          [imgBlank,imgBlank,imgBlank,imgBlank])
#           [imgInvGradeDisplay,imgFinal,imgBlank,imgBlank])
imgStacked=stackImages(imgArray,0.5)   #imgStacked=stackImages(imgArray,0.3,labels)

# cv2.imshow('dksfjd',nameCropped2)


# cv2.waitKey(0)

In [363]:
cv2.imshow('stacked imgs',imgStacked)
cv2.waitKey(0)

-1